In [108]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc

import time
import pandas as pd

In [124]:
numero = 5

In [109]:
options = uc.ChromeOptions()
options.add_argument("--no-sandbox")

driver = uc.Chrome(options=options)

In [110]:
url = 'https://www.filmaffinity.com/es/topgen.php'
driver.get(url)

In [111]:
# Meterse en la web
driver.get(url)

# Rechazar cookies
boton_rechazo_html = '//*[@id="disagree-btn"]'
boton_rechazo = driver.find_element(By.XPATH, boton_rechazo_html)
boton_rechazo.click()

In [112]:
n_reps = 250//30 + 1

In [113]:
for _ in range(n_reps):
    # Bajamos hasta abajo de la pagina
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    # Cogemos el boton y pulsamos
    boton_html = '//*[@id="load-more-bt"]/i'
    boton = driver.find_element(By.XPATH, boton_html)
    boton.click()
    # Esperamos
    time.sleep(2)

In [114]:
soup_original = bs(driver.page_source, 'lxml')

In [ ]:
links_peli = []
for i in range(numero):
    links_peli.append(soup_original.select('div.mc-title a')[i]['href'])

In [117]:
links_peli

['https://www.filmaffinity.com/es/film809297.html',
 'https://www.filmaffinity.com/es/film107937.html',
 'https://www.filmaffinity.com/es/film730528.html',
 'https://www.filmaffinity.com/es/film399474.html',
 'https://www.filmaffinity.com/es/film489970.html']

In [120]:
titulos = []
years = []
direcciones = []
repartos = []

for link in links_peli:
    driver.get(link)
    soup = bs(driver.page_source, 'lxml')
    time.sleep(5)

    todo = soup.select('#left-column')[0].get_text()
    todo = todo.split('\n')
    todo_limpio =  list(filter(lambda x: x != '', todo))

    index = todo_limpio.index('Título original')
    titulo = todo_limpio[index+1].strip()
    titulo_bueno = titulo.split('aka')[0]


    index = todo_limpio.index('Año')
    year = todo_limpio[index+1].strip()

    index = todo_limpio.index('Dirección')
    direccion = todo_limpio[index+1].strip()

    index = todo_limpio.index('Reparto')
    reparto= todo_limpio[index+1]

    titulos.append(titulo_bueno)
    years.append(year)
    direcciones.append(direccion)
    repartos.append(reparto)

    time.sleep(5)



In [127]:
nota = []
for i in range(numero):
    nota.append(soup_original.select('div.avg-rating')[i].get_text())

In [129]:
df = pd.DataFrame({
    'titulo': titulos,
    'years': years,
    'direccion': direcciones,
    'reparto': repartos,
    'nota': nota
})

df

,titulo,years,direccion,reparto,nota
0,The Godfather,1972,Francis Ford Coppola,Marlon BrandoAl PacinoJames CaanRobert DuvallD...,"9,0"
1,Planet Earth II,2016,"Elizabeth White, Justin Anderson, Ed Charles, ...",David Attenborough,"8,9"
2,The Godfather Part II,1974,Francis Ford Coppola,Al PacinoRobert De NiroDiane KeatonRobert Duva...,"8,9"
3,The Wire,2002,"David Simon (Creador), Joe Chappelle, Ernest R...",Dominic WestLance ReddickSonja SohnClarke Pete...,"8,8"
4,Breaking Bad,2008,"Vince Gilligan (Creador), Michelle MacLaren, A...",Bryan CranstonAaron PaulAnna GunnDean NorrisBe...,"8,8"
